## 1.  
Есть три сундука с драгоценными камнями.  
В первом сундуке лежат 5 рубинов и 3 изумруда, во втором – только рубины, а в третьем – только изумруды.  
Случайно выбирается один сундук и из него случайно вытягивается один камень. Какова вероятность того, что это рубин? Поясните свой ответ.

**Формула полной вероятности**

$$P(A) = P(B_1) \cdot P(A\:|\:B_1) + P(B_2) \cdot P(A\:|\:B_2) \; + ... + \; P(B_n) \cdot P(A\:|\:B_n)$$


Решим эту задачу с помощью формулы полной вероятности. Событием $A$ будем считать то, что достанут рубин, а событием $B$ — что выберут определенный сундук. Причем вероятность выбора одной из трех сундуков будет равна 1/3.

Вероятность достать рубин при условии, что выбран первый сундук, равен 5/(3 + 5), то есть 5/8. При выборе второго сундука вероятность вынуть рубин равна 1, так как там лежат все рубины. В случае с третьим сундуком вероятность равна 0, так как там все камни - изумруды.

$$P(A) = \frac{1}{3} \cdot \frac{5}{8} + \frac{1}{3} \cdot 1 + \frac{1}{3} \cdot 0 = \frac{5}{24} + 0 + \frac{1}{3} = \frac{13}{24}$$

In [1]:
P = 1/3 * 5/8 + 1/3 * 1 + 1/3 * 0
P

0.5416666666666666

## 2. 
Есть таблица с данными по балансу на счете пользователя за каждый день. Записи уникальны по ключу user_id + date.  
Дано: параметры date_min и date_max.  
Задача: для каждого пользователя найти все диапазоны между date_min и date_max включительно, где баланс был непрерывно больше нуля;   вывести длину диапазона и средний баланс для каждого диапазона.  
Пример: date_min = 01.05.2019; date_max = 05.05.2019

In [2]:
import pandas as pd
import numpy as np

### Обзор данных

* **user_id** - ID пользователя (varchar)
* **date** - Дата (date)
* **balance** - Баланс в рублях (int)

In [3]:
data = {
    
    'user_id': ['abc001', 'abc001', 'abc001', 'abc001', 'abc001', 'abc002', 'abc002', 'abc002', 'abc002', 'abc002'],
    'date': ['01.05.2019', '02.05.2019', '03.05.2019', '04.05.2019', '05.05.2019', '01.05.2019', '02.05.2019', '03.05.2019', '04.05.2019', '05.05.2019'],
    'balance': [0, 100, 50, 30, 0, 100, 50, 0, 0, 200]  
}

df = pd.DataFrame(data)
df

,user_id,date,balance
0,abc001,01.05.2019,0
1,abc001,02.05.2019,100
2,abc001,03.05.2019,50
3,abc001,04.05.2019,30
4,abc001,05.05.2019,0
5,abc002,01.05.2019,100
6,abc002,02.05.2019,50
7,abc002,03.05.2019,0
8,abc002,04.05.2019,0
9,abc002,05.05.2019,200


In [4]:
df['date'] = pd.to_datetime(df['date'], format = '%d.%m.%Y') 

In [5]:
def build_report(df, date_min=None, date_max=None):
    date_min = pd.to_datetime(date_min, format='%d.%m.%Y') if date_min else df['date'].min()
    date_max = pd.to_datetime(date_max, format='%d.%m.%Y') if date_max else df['date'].max()
    
    dfc = df.copy()
    dfc = dfc.loc[(dfc['date'] >= date_min) & (dfc['date'] <= date_max)] 
    
    dfc.sort_values(by=['user_id', 'date'], inplace=True, ignore_index=True)
    dfc['zero_cnt'] = (dfc['balance'] == 0).cumsum()
    
    report_df = dfc[dfc['balance'] != 0].groupby(['user_id', 'zero_cnt']).agg(
    date_start = ('date', 'min'),
    date_end = ('date', 'max'),
    length = ('balance', 'count'),
    balance_avg = ('balance', 'mean')).reset_index()
        
    return report_df

In [6]:
build_report(df)

,user_id,zero_cnt,date_start,date_end,length,balance_avg
0,abc001,1,2019-05-02,2019-05-04,3,60
1,abc002,2,2019-05-01,2019-05-02,2,75
2,abc002,4,2019-05-05,2019-05-05,1,200


In [7]:
build_report(df, date_min='04.05.2019', date_max='05.05.2019')

,user_id,zero_cnt,date_start,date_end,length,balance_avg
0,abc001,0,2019-05-04,2019-05-04,1,30
1,abc002,2,2019-05-05,2019-05-05,1,200
